In [1]:
%load_ext autoreload
%autoreload 2

import kagglehub
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import TensorDataset, DataLoader
import optuna
from modules import EEGDataset, split_and_get_loaders, evaluate_model

torch.device('cuda' if torch.cuda.is_available() else 'cpu')

/home/zeyadcode/.pyenv/versions/ai_venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cpu')

In [2]:
#! need to modify those for the competition itself
TRIAL_LENGTH = 640  # frequency of changing.. frequency
# Download dataset
# path_1 = kagglehub.dataset_download("xuannguyenuet2004/12-class-ssvep-eeg-data") proofed to be bad
path_1 = kagglehub.dataset_download("girgismicheal/steadystate-visual-evoked-potential-signals")
path_1 += "/SSVEP (BrainWheel)"
print("Download datasetaset files:", "\n", path_1)

Download datasetaset files: 
 /home/zeyadcode/.cache/kagglehub/datasets/girgismicheal/steadystate-visual-evoked-potential-signals/versions/1/SSVEP (BrainWheel)


In [6]:
class SSVEPClassifier(nn.Module):
    def __init__(self, input_size: int, out_size: int, hidden_size: int, num_layers: int, dropout: float, bidirectional: bool, device=None):
        super().__init__()
        if device is None:
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.device = device
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.dir_mult = 2 if bidirectional else 1

        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=dropout, bidirectional=bidirectional, device=self.device, batch_first=True)
        self.fc_out = nn.Sequential(
            nn.Linear(hidden_size * self.dir_mult, out_size),
        )

    def forward(self, x: torch.Tensor):
        h0 = torch.zeros([self.num_layers * self.dir_mult, x.shape[0], self.hidden_size], device=self.device)
        c0 = torch.zeros([self.num_layers * self.dir_mult, x.shape[0], self.hidden_size], device=self.device)

        out, (hn, cn) = self.lstm(x, (h0, c0))  # out shape [B x window_length x out_shape]
        return self.fc_out(out[:, -1])

In [7]:
class Trainer:
    def __init__(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.train_epochs = 1000
        self.tune_epochs = 5
        self.optuna_n_trials = 300

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = None
        self.trial = None

        self.train_loader = None
        self.eval_loader = None
        self.test_loader = None
        self.dataset = None

        self.storage = "sqlite:///optuna_studies.db"
        self.study_name = "ssvep_classifier_optimization"

    def _train_loop(self, n_epochs: int, should_print=False):
        assert isinstance(self.optimizer, torch.optim.Optimizer), "optimizer is not a valid optimizer"
        assert isinstance(self.train_loader, DataLoader), "train_laoder is not valid Datloader"
        assert isinstance(self.trial, optuna.Trial), "trial is not a valid optuna.Trial"

        self.model.train()
        for epoch in range(n_epochs):
            self.model.to(self.device)

            for x, y in self.train_loader:
                x = x.to(self.device)
                y = y.to(self.device)

                y_pred = self.model(x)  # B x out_size
                loss = self.criterion(y_pred, y)

                self.optimizer.zero_grad()
                loss.backward()
                self.optimizer.step()

            evaluation = evaluate_model(self.model, self.val_loader, self.device)
            self.trial.report(evaluation, epoch)
            if self.trial.should_prune():
                optuna.exceptions.TrialPruned()

            if should_print:
                print(f"epoch {epoch}, evaluation {evaluation}")

    def objective(self, trial: optuna.Trial):
        self.trial = trial

        window_length = self.trial.suggest_categorical("window_length", [80, 128, 160])
        stride_factor = self.trial.suggest_int("stride", 1, 4)
        stride = window_length // stride_factor
        
        self.dataset = EEGDataset(path_1, TRIAL_LENGTH, window_length, stride=stride)
        unique_freqs = torch.unique(self.dataset.labels)

        input_size = self.dataset.data[0].shape[1]  # of shape C x T, get the T out we care about it
        out_size = len(unique_freqs)

        hidden_size = self.trial.suggest_int("hidden_size", 32, 256, step=16)
        num_layers = self.trial.suggest_int("num_layers", 1, 16)
        dropout = self.trial.suggest_float("dropout", 0, 0.5)  # not sure what is the default step here tho
        bidirectional = self.trial.suggest_categorical("bidirectional", [True, False])
        lr = self.trial.suggest_float("lr", 1e-5, 0.1, log=True)
        batch_size = self.trial.suggest_categorical("batch_size", [16, 32, 64, 128])

        self.model = SSVEPClassifier(input_size, out_size, hidden_size, num_layers, dropout, bidirectional)
        self.train_loader, self.val_loader, self.test_loader = split_and_get_loaders(self.dataset, batch_size)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr)
        
        self._train_loop(self.tune_epochs)

        evaluation = evaluate_model(self.model, self.val_loader, self.device)
        return evaluation

    def _get_study(self):
        return optuna.create_study(study_name=self.study_name, storage=self.storage, direction="maximize", load_if_exists=True)

    def optimize(self, delete_existing=False):
        if delete_existing:
            optuna.delete_study(study_name=self.study_name, storage=self.storage)

        study = self._get_study()
        study.optimize(self.objective, n_trials=self.optuna_n_trials, timeout=60 * 10)

        # Print optimization results
        print("\nStudy statistics:")
        print(f"  Number of finished trials: {len(study.trials)}")
        print(f"  Number of pruned trials: {len(study.get_trials(states=[optuna.trial.TrialState.PRUNED]))}")
        print(f"  Number of complete trials: {len(study.get_trials(states=[optuna.trial.TrialState.COMPLETE]))}")

        print("\nBest trial:")
        trial = study.best_trial
        print(f"  Value: {trial.value}")
        print("\nBest hyperparameters:")
        for key, value in trial.params.items():
            print(f"  {key}: {value}")

        return study.best_params


trainer = Trainer()
trainer.optimize(delete_existing=True)

[I 2025-06-16 10:52:14,541] A new study created in RDB with name: ssvep_classifier_optimization
[W 2025-06-16 10:52:21,547] Trial 0 failed with parameters: {'window_length': 128, 'stride': 2, 'hidden_size': 96, 'num_layers': 16, 'dropout': 0.38837191960090583, 'bidirectional': False, 'lr': 0.003908028647578137, 'batch_size': 64} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "/home/zeyadcode/.pyenv/versions/ai_venv/lib/python3.12/site-packages/optuna/study/_optimize.py", line 197, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_478722/2547561819.py", line 73, in objective
    self._train_loop(self.tune_epochs)
  File "/tmp/ipykernel_478722/2547561819.py", line 34, in _train_loop
    y_pred = self.model(x)  # B x out_size
             ^^^^^^^^^^^^^
  File "/home/zeyadcode/.pyenv/versions/ai_venv/lib/python3.12/site-packages/torch/nn/modules/module.py", line 1751, in _wrapped_call_imp

KeyboardInterrupt: 